In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
sc.stop()

## 라이브러리
- 주요 라이브러리 불러오기

In [4]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pyspark

print(np.__version__)
print(pd.__version__)
print(matplotlib.__version__)
print(sns.__version__)
print(sklearn.__version__)
print(pyspark.__version__)

1.23.4
1.5.1
3.6.2
0.12.1
1.1.3
3.2.0


## Spark 세션 만들기 (평가1)
- master : "local[1]"
- appName : my1stSpark

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("my1stSpark").getOrCreate()
spark

22/11/08 09:41:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## RDD의 작동원리
- 데이터 --> Transformation 메서드 활용 (lambda 함수 활용) --> Action 메서드 활용
- PairRDD : 값이 Dictionary + 튜플처럼 저장됨

In [7]:
# Dictionary
temp_dict = {
    "name" : ["A", "B", "C"]
}
temp_dict["name"]

['A', 'B', 'C']

In [8]:
# Tuple
temp_tuple = (1,2,3,4)
temp_tuple

(1, 2, 3, 4)

In [9]:
# 프로그래밍 수강생 조사 (PairRDD 형태)
data = [('python', 10),('javascript',5),('java',20),('python', 5),('R',5),('java',10)]
data

[('python', 10),
 ('javascript', 5),
 ('java', 20),
 ('python', 5),
 ('R', 5),
 ('java', 10)]

## PairRDD 생성

In [13]:
regi_lan = spark.sparkContext.parallelize(data)
print(regi_lan)
print(type(regi_lan))

# Action Method
print(regi_lan.collect())

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:274
<class 'pyspark.rdd.RDD'>
[('python', 10), ('javascript', 5), ('java', 20), ('python', 5), ('R', 5), ('java', 10)]


### PairRDD 메서드 종류
- 참조 : https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations --> PairRDD 찾기

### groupbyKey()
- 참조 : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.groupByKey.html#pyspark.RDD.groupByKey 

In [14]:
group_rdd = regi_lan.groupByKey().collect()
for keys, values in group_rdd :
    print(keys, "-->", list(values))

python --> [10, 5]
javascript --> [5]
java --> [20, 10]
R --> [5]


In [15]:
sorted(regi_lan.groupByKey().mapValues(len).collect())

[('R', 1), ('java', 2), ('javascript', 1), ('python', 2)]

In [16]:
sorted(regi_lan.groupByKey().mapValues(list).collect())

[('R', [5]), ('java', [20, 10]), ('javascript', [5]), ('python', [10, 5])]

### sortByKey(
- 참조 : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sortByKey.html?highlight=sortbykey

In [17]:
print(regi_lan.sortByKey().collect())

[('R', 5), ('java', 20), ('java', 10), ('javascript', 5), ('python', 10), ('python', 5)]


In [18]:
print(regi_lan.sortByKey(ascending=False).collect())

[('python', 10), ('python', 5), ('javascript', 5), ('java', 20), ('java', 10), ('R', 5)]


### reduceByKey()
- 사칙연산
- lambda 함수 사용 필요
- 참조 : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html?highlight=reducebykey

In [19]:
regi_lan.reduceByKey(lambda x1, x2 : x1+x2).collect()

[('python', 15), ('javascript', 5), ('java', 30), ('R', 5)]

In [20]:
regi_lan.reduceByKey(lambda x1, x2 : x1-x2).collect()

[('python', 5), ('javascript', 5), ('java', 10), ('R', 5)]

In [21]:
regi_lan.reduceByKey(lambda x1, x2 : x1*x2).collect()

[('python', 50), ('javascript', 5), ('java', 200), ('R', 5)]

In [22]:
regi_lan.reduceByKey(lambda x1, x2 : x1/x2).collect()

[('python', 2.0), ('javascript', 5), ('java', 2.0), ('R', 5)]